# Calculate $\omega \text{s}$ and corresponding eigenvectors

In [4]:
from mlat.const import dirac, pauli
import numpy as np
import numpy.linalg as la

u = 0.
Nx = 10
Ny = 10
m = 1.

d = dirac(Nx, Ny)
p = pauli()

H = 0

for mx in range(1, Nx-2):
    for my in range(1, Ny-1):
        fst =  np.kron(d.ket(mx+1, my).dot(d.bra(mx, my)), (p.z + 1.j * p.x) / 2.)
        H += fst + fst.conj().T
for mx in range(1, Nx-1):
    for my in range(1, Ny-2):
        fst =  np.kron(d.ket(mx, my+1).dot(d.bra(mx, my)), (p.z + 1.j * p.y) / 2.)
        H += fst + fst.conj().T
for mx in range(1, Nx-1):
    for my in range(1, Ny-1):
        fst =  np.kron(d.ket(mx, my).dot(d.bra(mx, my)), p.z)
        H += u * fst
# Fixed points
for my in range(Ny):
    H += np.kron(d.ket(1, my).dot(d.bra(0, my)), (p.z + 1.j * p.x) / 2.)
for my in range(Ny):
    H += np.kron(d.ket(Nx-1, my).dot(d.bra(Nx-2, my)), (p.z + 1.j * p.x) / 2.)
for mx in range(Nx):
    H +=  np.kron(d.ket(mx, 1).dot(d.bra(mx, 0)), (p.z + 1.j * p.y) / 2.)
for mx in range(Nx):
    H +=  np.kron(d.ket(mx, Ny-2).dot(d.bra(mx, Ny-1)), (p.z + 1.j * p.y) / 2.)


M = np.diag([m] * H.shape[0])


evals, evecs = la.eig(la.inv(M).dot(H))
idcs = np.argsort(evals)
evals, evecs = evals[idcs], evecs[idcs]

ws = np.sqrt(evals).real

print(np.argmin(np.abs(ws-1)))
# print(np.where(ws 3.27458412e-01))
print(evecs.shape)
# ws[500]
print(ws)

154
(200, 200)
[7.44118289e-18 1.43061829e-16 6.30855651e-17 1.99715975e-17
 1.63888088e-16 7.12718505e-17 4.95319348e-20 2.02654279e-17
 3.75608806e-16 4.28215784e-17 1.19227884e-16 1.94558694e-17
 3.90741845e-16 4.34912859e-17 7.24557429e-17 5.36244329e-17
 8.72063123e-17 1.05265992e-17 9.55865695e-17 4.57600034e-17
 1.09489864e-17 1.65215533e-17 5.91955886e-17 1.84963039e-17
 1.57125469e-17 1.85129427e-17 3.47493881e-17 2.10622918e-17
 1.18376317e-16 1.78803518e-17 1.49426419e-16 9.50404984e-17
 9.31868359e-17 1.21549821e-17 2.56450596e-17 1.54997931e-16
 4.01137597e-16 5.05417821e-17 1.38067415e-16 6.68765640e-17
 5.43619861e-17 4.44731270e-18 1.20547984e-17 9.10291028e-20
 1.82477479e-16 2.35843373e-17 3.54077374e-17 1.93414228e-17
 5.86773423e-17 9.42877596e-17 5.45385122e-17 1.32627991e-16
 4.46889671e-17 1.81665306e-16 1.31624243e-16 7.72481354e-17
 6.98215986e-17 4.36288529e-17 1.57594676e-16 3.56654242e-19
 1.18792363e-17 2.73834892e-18 4.42036938e-17 5.81629556e-17
 0.000000

# Animation

In [5]:
import numpy as np
import plotly.graph_objects as go

# N=10 -78
mode = 154
w = ws[mode] 

grid_cell_size = 2

grid = np.array([[[grid_cell_size * i, grid_cell_size * j] for i in range(Nx)] for j in range(Ny)], dtype=np.complex128)

# Print infos
print("Angular frequency: ", w)

# Parameters for animation
fps = 30 # /s
animation_length = 10 # s

dgrid = np.zeros(grid.shape, dtype=np.complex128)

for mx in range(Nx):
    for my in range(Ny):
        dgrid[my, mx] = evecs[mode].dot(np.kron(d.ket(mx, my), np.eye(2)))
#         dgrid[my, mx] = evecs[mode].dot(np.kron(np.eye(2), d.ket(mx, my)))
        



# Construct frames
frames = []
for t in range(int(animation_length * fps)):
    dt = t / fps
    dphase = dt * w * 2 * np.pi
    current_grid = grid + dgrid * np.exp(1.j * dphase)
    x = current_grid[:, :, 0].flatten()
    y = current_grid[:, :, 1].flatten()
    frames.append(go.Frame(data=[go.Scatter(x=x.real, y=y.real, mode='markers')]))

# Figure components
start_button = dict(
        label="Play",
        method="animate",
        args=[
            None, 
            {
                "frame":{"duration": 1000 / fps, "redraw": False},
                "fromcurrent": True, 
                "transition": {"duration": 100}
            }])
pause_button = dict(
        label="Pause",
        method="animate",
        args=[
            [None], 
            {
                "frame": {"duration": 0, "redraw": False},
                "mode": "immediate",
                "transition": {"duration": 0}
            }])

# Plot
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title="Dispersion relation animation",
        yaxis=dict(scaleanchor="x", scaleratio=1, autorange='reversed'),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[start_button, pause_button
            ])
        ]
    ),
    frames=frames[1:])
fig.show()

Angular frequency:  1.0076897673408196
